### Part I. ETL pipeline: pre-processing files

In [1]:
import pandas as pd
from cassandra.cluster import Cluster
import os
import glob
import csv

#### List of filepaths to process raw csv data files (event_data)

In [2]:
print(os.getcwd())

# current dir of event data
filepath = os.getcwd() + '/event_data'

# create list of files & collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print('\nTotal csv files: ', len(file_path_list))

/Users/abreu/Developer/GitHub/sparkify-cassandra

Total csv files:  30


#### Process the files to create the *csv* that will be used as data source for Casssandra tables

In [3]:
full_data_rows_list = [] 
    
# read data in every csv in list & append to list
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        # create a csv reader object
        csvreader = csv.reader(csvfile)
        # skip header
        next(csvreader)

        for line in csvreader:
            full_data_rows_list.append(line)
            
# total number of rows
print('\nTotal data rows: ', len(full_data_rows_list))


# create event_datafile_new.csv as source data for Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding='utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist', 'first_name', 'gender', 'item_in_session', 'last_name', 'length', 'level', 'location',
                     'session_id', 'song', 'user_id'])

    # write valid song data (artist & nextsong related records) into csv
    for row in full_data_rows_list:
        if row[0] == '':
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('\nTotal valid data rows: ', sum(1 for line in f))


Total data rows:  8056

Total valid data rows:  6821


### Part II. Apache Cassandra code

#### CSV file - <font color=red>*event_datafile_new.csv*</font> - contains the columns:
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

Note: Screenshot of denormalized data in <font color=red>*event_datafile_new.csv*</font> after running code above:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creat Cluster

In [4]:
# connect to a Cassandra instance (local machine: 127.0.0.1)
try:
    cluster = Cluster(['127.0.0.1'], port=9042)
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = {
            'class': 'SimpleStrategy',
            'replication_factor': 1
        }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Create tables to run the following 3 queries

#### <font color=green> 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4 </font>

#### <font color=blue> 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 </font>

#### <font color=orange> 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' </font>

#### File (csv): source of data

In [7]:
file = 'event_datafile_new.csv'
df = pd.read_csv(file, encoding='utf8')

#### Query 1 needs table: session_metadata

In [8]:
create_query_1 = "CREATE TABLE IF NOT EXISTS session_metadata "
create_query_1 = create_query_1 + "(session_id INT, item_in_session INT, artist TEXT, song TEXT,  length DECIMAL, " \
    "PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(create_query_1)
except Exception as e:
    print(e)

In [9]:
# load data into session_metadata
for i, row in df.iterrows():
    insert_query = "INSERT INTO session_metadata (session_id, item_in_session, artist, song, length) "
    insert_query = insert_query + "VALUES (%s, %s, %s, %s, %s)"
    try:
        session.execute(insert_query, (row.session_id, row.item_in_session, row.artist, row.song, row.length))
    except Exception as e:
        print(e)

In [10]:
select_query = "SELECT artist, song, length FROM session_metadata WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)

data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns=['artist', 'song', 'length'])


,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


#### Query 2 needs table: user_metadata

In [11]:
create_query_2 = "CREATE TABLE IF NOT EXISTS user_metadata "
create_query_2 = create_query_2 + "(user_id INT, session_id INT, item_in_session INT, artist TEXT, song TEXT, " \
                                  "first_name TEXT, last_name TEXT, " \
                                  "PRIMARY KEY (user_id, session_id, item_in_session))"

try:
    session.execute(create_query_2)
except Exception as e:
    print(e)

In [12]:
# load data into user_metadata
for i, row in df.iterrows():
    insert_query = "INSERT INTO user_metadata (user_id, session_id, item_in_session, artist, song, first_name, last_name) "
    insert_query = insert_query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    try:
        session.execute(insert_query, (row.user_id, row.session_id, row.item_in_session, row.artist, row.song,
                                       row.first_name, row.last_name))
    except Exception as e:
        print(e)

In [13]:
select_query = "SELECT artist, song, first_name, last_name, item_in_session FROM user_metadata WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)

data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns=['artist', 'song', 'first_name', 'last_name', 'item_in_session'])

,artist,song,first_name,last_name,item_in_session
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0
1,Three Drives,Greece 2000,Sylvie,Cruz,1
2,Sebastien Tellier,Kilometer,Sylvie,Cruz,2
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3


#### Query 3 needs table: song_metadata

In [14]:
create_query_3 = "CREATE TABLE IF NOT EXISTS song_metadata "
create_query_3 = create_query_3 + "(song TEXT, user_id INT, first_name TEXT, last_name TEXT, PRIMARY KEY (song, user_id))"

try:
    session.execute(create_query_3)
except Exception as e:
    print(e)

In [15]:
# load data into song_metadata
for i, row in df.iterrows():
    insert_query = "INSERT INTO song_metadata (song, user_id, first_name, last_name) "
    insert_query = insert_query + "VALUES (%s, %s, %s, %s)"
    try:
        session.execute(insert_query, (row.song, row.user_id, row.first_name, row.last_name))
    except Exception as e:
        print(e)


In [16]:
select_query = "SELECT song, first_name, last_name FROM song_metadata WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)

data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns=['song', 'first_name', 'last_name'])

,song,first_name,last_name
0,All Hands Against His Own,Jacqueline,Lynch
1,All Hands Against His Own,Tegan,Levine
2,All Hands Against His Own,Sara,Johnson


#### Drop tables

In [17]:
session_metadata_drop = "DROP TABLE IF EXISTS session_metadata"
user_metadata_drop = "DROP TABLE IF EXISTS user_metadata"
song_metadata_drop = "DROP TABLE IF EXISTS song_metadata"

drop_table_queries = [session_metadata_drop, user_metadata_drop, song_metadata_drop]

for query in drop_table_queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)


#### Close session & connection¶

In [18]:
session.shutdown()
cluster.shutdown()